<a href="https://colab.research.google.com/github/BushraAlmutiri/MedicalInsurance/blob/main/gp8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from flask import Flask, request, jsonify
import sqlite3
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import pickle
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
app = Flask(__name__)
conn = sqlite3.connect('insurance.db')
c = conn.cursor()

In [ ]:
def load_dataset():
    df = pd.read_csv("insurance.csv")
    return df

In [ ]:
def preprocess_data(df):
    df['sex'] = df['sex'].apply({'male': 0, 'female': 1}.get)
    df['smoker'] = df['smoker'].apply({'yes': 1, 'no': 0}.get)
    df['region'] = df['region'].apply({'southwest': 1, 'southeast': 2, 'northwest': 3, 'northeast': 4}.get)
    return df

In [ ]:
def train_models(X_train, y_train):
    linear_regression_model = LinearRegression()
    svr_model = SVR()
    decision_tree_model = DecisionTreeRegressor()

    linear_regression_model.fit(X_train, y_train)
    svr_model.fit(X_train, y_train)
    decision_tree_model.fit(X_train, y_train)

    return linear_regression_model, svr_model, decision_tree_model

In [ ]:
def make_predictions(models, input_data):
    linear_regression_prediction = models[0].predict(input_data)[0]
    svr_prediction = models[1].predict(input_data)[0]
    decision_tree_prediction = models[2].predict(input_data)[0]

    return linear_regression_prediction, svr_prediction, decision_tree_prediction

In [ ]:
def determine_best_model(mse_values):
    linear_regression_mse, svr_mse, decision_tree_mse = mse_values

    if linear_regression_mse <= svr_mse and linear_regression_mse <= decision_tree_mse:
        return 'Linear Regression'
    elif svr_mse <= linear_regression_mse and svr_mse <= decision_tree_mse:
        return 'Support Vector Regression'
    else:
        return 'Decision Tree'

In [ ]:
c.execute('''CREATE TABLE IF NOT EXISTS predictions
             (age INTEGER, sex INTEGER, bmi REAL, children INTEGER, smoker INTEGER, region INTEGER,
             linear_regression_prediction REAL, svr_prediction REAL, decision_tree_prediction REAL, best_model TEXT)''')

In [ ]:
def insert_predictions(input_values, predictions, best_model):
    c.execute("INSERT INTO predictions VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
              (input_values[0], input_values[1], input_values[2], input_values[3], input_values[4], input_values[5],
               predictions[0], predictions[1], predictions[2], best_model))
    conn.commit()

In [ ]:
def perform_prediction(input_data):
    input_values = [input_data['age'], input_data['sex'], input_data['bmi'], input_data['children'], input_data['smoker'], input_data['region']]
    input_data = np.array([input_values])

    return input_data

In [ ]:
linear_regression_model, svr_model, decision_tree_model = None, None, None

In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    input_data = request.get_json()
    input_data_array = perform_prediction(input_data)

In [ ]:
with open('linear_regression_model.pkl', 'wb') as f:
        pickle.dump(linear_regression_model, f)

with open('svr_model.pkl', 'wb') as f:
        pickle.dump(svr_model, f)

with open('decision_tree_model.pkl', 'wb') as f:
        pickle.dump(decision_tree_model, f)

app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
def load_linear_regression_model():
    try:
        with open('linear_regression_model.pkl', 'rb') as f:
            linear_regression_model = pickle.load(f)
    except Exception as e:
        print("Error loading linear regression model:", e)
        return jsonify({"error": "Failed to load linear regression model"})

def load_svr_model():
    try:
        with open('svr_model.pkl', 'rb') as f:
            svr_model = pickle.load(f)
    except Exception as e:
        print("Error loading SVR model:", e)
        return jsonify({"error": "Failed to load SVR model"})

def load_decision_tree_model():
    try:
        with open('decision_tree_model.pkl', 'rb') as f:
            decision_tree_model = pickle.load(f)
    except Exception as e:
        print("Error loading Decision Tree model:", e)
        return None

In [ ]:
def check_model_load_status(linear_regression_model, svr_model, decision_tree_model):
    if linear_regression_model is None or svr_model is None or decision_tree_model is None:
        return True
    else:
        return False

In [ ]:
def extract_input_features(input_data):
    input_values = [input_data['age'], input_data['sex'], input_data['bmi'], input_data['children'], input_data['smoker'], input_data['region']]
    input_data_array = np.array([input_values])
    return input_data_array

In [ ]:
input_data_array = np.array([[10, 100]])

In [ ]:
linear_regression_prediction = linear_regression_model.predict(input_data_array)[0]
svr_prediction = svr_model.predict(input_data_array)[0]
decision_tree_prediction = decision_tree_model.predict(input_data_array)[0]

In [ ]:
mse_values = (
        mean_squared_error(y_test, linear_regression_predictions),
        mean_squared_error(y_test, svr_predictions),
        mean_squared_error(y_test, decision_tree_predictions)
    )

    best_model = determine_best_model(mse_values)


    insert_predictions(input_values, (linear_regression_prediction, svr_prediction, decision_tree_prediction), best_model)

    return jsonify(linear_regression_prediction)

In [ ]:
mse_values = (
    mean_squared_error(y_test, linear_regression_predictions),
    mean_squared_error(y_test, svr_predictions),
    mean_squared_error(y_test, decision_tree_predictions)
)

best_model = determine_best_model(mse_values)

insert_predictions(input_values, (linear_regression_prediction, svr_prediction, decision_tree_prediction), best_model)

return jsonify(linear_regression_prediction)

In [ ]:
if __name__ == '__main__':
    df = load_dataset()
    df = preprocess_data(df)

    X = df[['age', 'sex', 'bmi', 'children', 'smoker', 'region']]
    y = df['charges']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    linear_regression_model, svr_model, decision_tree_model = train_models(X_train, y_train)